In [ ]:
import matplotlib
matplotlib.use('Agg')

# import package
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from pyimagesearch.nn.conv import MiniVGGNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
args = {
    'output': '',
    'models': '',
    'num_models' : 5,

}

In [ ]:
# load training data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float') / 255.0
testX = testX.astype('float')

# convert label from int to vec
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit(testY)

# init the label names for cifar-10
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [ ]:
# apply data augmentation
aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, fill_mode='nearest')

In [ ]:
# loop over the number of models to train
for i in np.arange(0, args['num_models']):
    # init optimizers and model
    print("[INFO] training model {}/{}".format(i + 1, args['num_models']))
    opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nestorov=True)
    model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # train the network
    H = model.fit_generator(aug.flow(trainX, trainY, batch_size=64),
    validation_data=(testX,testY), epochs=40,
    steps_per_epoch=len(trainY) // 64, verbose=1)

    # save model to disk
    p = [args['models'], "model_{}.model".format(i)]
    model.save(os.path.sep.join(p))

    # classification reprot
    predictions = model.predict(testX, batch_size=64)
    report =classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=labelNames)

    # save the c-report
    p = [args['output'], "model_{}.txt".format(i)]
    f = open(os.path.sep.join(p), 'w')
    f.write(report)
    f.close()

    
    # plot the training loss and accuracy
    p = [args["output"], "model_{}.png".format(i)]
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, 40), H.history["loss"],
    label="train_loss")
    plt.plot(np.arange(0, 40), H.history["val_loss"],
    label="val_loss")
    plt.plot(np.arange(0, 40), H.history["accuracy"],
    label="train_acc")
    plt.plot(np.arange(0, 40), H.history["val_accuracy"],
    label="val_acc")
    plt.title("Training Loss and Accuracy for model {}".format(i))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig(os.path.sep.join(p))
    plt.close()